In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import math
import threading
from nltk.corpus import wordnet
import time
from multiprocessing import Process, Queue
import multiprocessing
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('punkt')
import csv 
import requests 
import xml.etree.ElementTree as ET 
import os
import pandas as pd
import re
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from multiprocessing import Pool
import time
import random
from sklearn.preprocessing import OneHotEncoder
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
import random
from nltk.stem.porter import *
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from multiprocessing import Pool, Manager
from datetime import datetime

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
!unzip XML-Files.zip -d XML-Files

Archive:  XML-Files.zip
  inflating: XML-Files/XML-Files/AI_Meta_Posts.xml  
  inflating: XML-Files/XML-Files/AI_Posts.xml  
  inflating: XML-Files/XML-Files/Computer_graphics_Posts.xml  
  inflating: XML-Files/XML-Files/Computergraphics_Meta_Posts.xml  
  inflating: XML-Files/XML-Files/CS_Meta_Posts.xml  
  inflating: XML-Files/XML-Files/CS_Posts.xml  
  inflating: XML-Files/XML-Files/DataScience_Meta_Posts.xml  
  inflating: XML-Files/XML-Files/DataScience_Posts.xml  


In [4]:
fields = ['Id', 'Text', 'Topic']

def parseXML(xmlfile, start_count): 
    """This function would parse XML file and would extract important information from xml tag 'text' 
    and returns list containing dict whcih would have text"""
    #https://www.geeksforgeeks.org/xml-parsing-python/
    #create element tree object 
    print("File", xmlfile)
    tree = ET.parse(xmlfile) # Creates a parse tree like https://www.google.com/search?q=xml+parse+tree&sxsrf=ALeKk00rw2XjRSAi-qGfaA0Rg4EtJzyx_Q:1585918970675&source=lnms&tbm=isch&sa=X&ved=2ahUKEwi0wY7HqMzoAhVXAXIKHYfdDrkQ_AUoAXoECA4QAw&biw=1366&bih=657#imgrc=P4mJt0kjVEuqVM
    topic = xmlfile.split("/content/XML-Files")[1].split("_")[0] # Getting xml file_name from string containing directory/file_name.xml 
    # get root element using which we would parse xml tree
    root = tree.getroot() 
    # create empty list for news items 
    newsitems = [] 
    count = start_count
    # iterate news items 
    for each_row in root.iter("row"):#  iterates only over those elements that have the required tag:"row" 
        news = {}
        news["Id"] = count
        news["Text"] = each_row.attrib["Body"]
        news["Topic"] = topic
        count=count+1
        newsitems.append(news) #[{id:1,text:" Some text  ",Topic : "AI"},{{id:1,text:" Some text  ",Topic : "AI"},{{id:1,text:" Some text  ",Topic : "AI"},...........]
    # return news items list 
    print("len", len(newsitems))
    return newsitems # To save this dict on csv we are putting dict into a list https://www.tutorialspoint.com/How-to-save-a-Python-Dictionary-to-CSV-file

In [5]:
def savetoCSV(newsitems, filename): 
    """Writes the list of dict returned from above function onto a csv file""" 
    # writing to csv file 
    with open(filename, 'w', encoding = 'utf-8') as csvfile: 
        # creating a csv dict writer object 
        writer = csv.DictWriter(csvfile, fieldnames = fields) 
        # writing headers (field names) 
        writer.writeheader() 
        # writing data rows 
        writer.writerows(newsitems)

In [6]:
def filterpostfiles(filename):
    """Returns the name of  xml files present in the directory CML Files """
    return filename.endswith("Posts.xml") 
postfiles = filter(filterpostfiles, os.listdir("/content/XML-Files"))

# specifying the fields for csv file 
fields = ['Id', 'Text', 'Topic']  
start_count = 0
for each_file in postfiles:
    print(each_file)
    # parse xml file 
    newsitems = parseXML("/content/XML-Files/"+each_file, start_count)
    #Get only name of xml file by sepearting out extension use the same name to save file as .csv 
    csv_filename = each_file.split('.')[0] + ".csv"
    print("csv_filename", csv_filename)
    # store news items in a csv file 
    savetoCSV(newsitems, "/content/CSV-Files/" + csv_filename)
    start_count = len(newsitems) + start_count

DataScience_Posts.xml
File /content/XML-Files/DataScience_Posts.xml
len 54869
csv_filename DataScience_Posts.csv
CS_Posts.xml
File /content/XML-Files/CS_Posts.xml
len 81429
csv_filename CS_Posts.csv
AI_Posts.xml
File /content/XML-Files/AI_Posts.xml
len 16760
csv_filename AI_Posts.csv
AI_Meta_Posts.xml
File /content/XML-Files/AI_Meta_Posts.xml
len 654
csv_filename AI_Meta_Posts.csv
CS_Meta_Posts.xml
File /content/XML-Files/CS_Meta_Posts.xml
len 1585
csv_filename CS_Meta_Posts.csv
Computer_graphics_Posts.xml
File /content/XML-Files/Computer_graphics_Posts.xml
len 6070
csv_filename Computer_graphics_Posts.csv
Computergraphics_Meta_Posts.xml
File /content/XML-Files/Computergraphics_Meta_Posts.xml
len 299
csv_filename Computergraphics_Meta_Posts.csv
DataScience_Meta_Posts.xml
File /content/XML-Files/DataScience_Meta_Posts.xml
len 509
csv_filename DataScience_Meta_Posts.csv


In [9]:
def filtercsvpostfiles(filename):
    """ Returns all the csv file name extracted from XML which are stored in CSV files"""
    return filename.endswith("Posts.csv") 
csvpostfiles = filter(filtercsvpostfiles, os.listdir("CSV-Files"))

In [10]:

total_dataframe = pd.DataFrame(columns=fields)
for each_file in csvpostfiles:
    #Merge multiple csv file created from each xml file into just one xml file
    print("each file", each_file)
    df = pd.read_csv("CSV-Files/"+ each_file)
    print("shape", df.shape)
    total_dataframe = total_dataframe.append(df, ignore_index=True)

each file CS_Meta_Posts.csv
shape (1585, 3)
each file AI_Posts.csv
shape (16760, 3)
each file DataScience_Meta_Posts.csv
shape (509, 3)
each file Computergraphics_Meta_Posts.csv
shape (299, 3)
each file AI_Meta_Posts.csv
shape (654, 3)
each file Computer_graphics_Posts.csv
shape (6070, 3)
each file CS_Posts.csv
shape (81429, 3)
each file DataScience_Posts.csv
shape (54869, 3)


In [11]:

print("Shape of total dataframe", total_dataframe.shape)

Shape of total dataframe (162175, 3)


In [12]:
total_dataframe = total_dataframe[~total_dataframe['Text'].isna()]
total_dataframe = total_dataframe[~total_dataframe['Text'].isnull()]
print("Shape of total dataframe after null value remove", total_dataframe.shape)

Shape of total dataframe after null value remove (161423, 3)


In [1]:

total_dataframe.to_pickle("QuestionsDataSet.pkl")

NameError: ignored